In [1]:
from taxcalc import *
import numpy as np
import pandas as pd
from pandas import DataFrame
import copy

In [2]:
# settings
# Change year here if running simulations for any other year
sim_year = 2017
agi_brackets = [(-1e14, 50000), (50000, 100000), (100000, 1e14)]
# list of dictionaries for each elasticity
elasticities = [
    {'lower': -0.5, 'middle': -0.5, 'upper': -0.5},
    {'lower': -1.0, 'middle': -1.0, 'upper': -1.0},
    {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}]

In [3]:
# lists for each output
base_list = list()
std_list = list()
charitable_list = list()
lower_mtr_list = list()
std_mtr_list = list()
std_charitable_list = list()
all_three_list = list()

In [4]:
# Reform dictionaries

# 2: raise standard deduction
std_reform = {
    sim_year: {
        '_STD': [[11000, 22000, 11000, 11000, 22000]]
    }
}

# 3: allow non-itemizers charitable deduction
charitable_reform = {
    sim_year: {
        '_STD_allow_charity_ded_nonitemizers': [True]
    }
}

# 4: lower top MTR to 35%
mtr_reform = {
    sim_year: {
        '_II_rt7': [0.35]
    }
}

In [5]:
# average charitable contributions by AGI level
contribs = pd.Series([342.96, 857.90, 1585.56])
recs = Records()
pol = Policy()

# blow-up contributions to 2013 levels
for year in range(2010, 2014):
    contribs *= recs.gfactors.factor_value('ATXPY', year)
contribs

0     389.598564
1     974.564405
2    1801.177687
dtype: float64

In [6]:
# intitalize calculator - used to assign charitable giving to non-itemizers
calc = Calculator(records=recs, policy=pol)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [7]:
# assign contribution amounts to non-itemizers based on AGI
calc.records.e19800 = np.where((calc.records.c00100 < 50000) & (calc.records.c04470 == 0),
                               contribs[0], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 >= 50000) & (calc.records.c00100 < 100000) &
                               (calc.records.c04470 == 0),
                               contribs[1], calc.records.e19800)
calc.records.e19800 = np.where((calc.records.c00100 > 100000) & (calc.records.c04470 == 0),
                               contribs[2], calc.records.e19800)
charitable_array = copy.deepcopy(calc.records.e19800)

In [8]:
# generate output table
# Total charitable giving only consists of cash contributions. Non-cash can be easily added

def table(calc_table):
    
    dictionary = {'Revenue ($m)': [],
                  'Total Giving ($m)': [], 
                  'Total Giving: Itemizers ($m)': [], 
                  'Total Giving: Non-Itemizers ($m)': [], 
                  'Number of Itemizers (m)': [], 
                  'Number of Non-Itemizers (m)': []}
    
    # Loop through each AGI income bin to get values
    for item in agi_brackets:
        
        val = calc_table.records._combined[(calc_table.records.c00100 >= item[0]) &
                                           (calc_table.records.c00100 < item[1])]
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1])]
        
        dictionary['Revenue ($m)'].append(val.sum() / 1000000)
        
        val = (calc_table.records.e19800[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1])] +
               calc_table.records.e20100[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1])])
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1])]
        
        dictionary['Total Giving ($m)'].append(float(val.sum()) / 1000000)
        
        val = (calc_table.records.e19800[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc.records.c04470 > 0)] +
               calc_table.records.e20100[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc.records.c04470 > 0)])
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1]) &
                                       (calc.records.c04470 > 0)]
        
        dictionary['Total Giving: Itemizers ($m)'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                      (calc_table.records.c00100 < item[1]) &
                                      (calc.records.c04470 > 0)]
        
        dictionary['Number of Itemizers (m)'].append(float(val.sum()) / 1000000)

        val = (calc_table.records.e19800[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc.records.c04470 == 0)] +
               calc_table.records.e20100[(calc_table.records.c00100 >= item[0]) &
                                         (calc_table.records.c00100 < item[1]) &
                                         (calc.records.c04470 == 0)])
        
        val *= calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                       (calc_table.records.c00100 < item[1]) &
                                       (calc.records.c04470 == 0)]
        
        dictionary['Total Giving: Non-Itemizers ($m)'].append(float(val.sum()) / 1000000)
        
        val = calc_table.records.s006[(calc_table.records.c00100 >= item[0]) &
                                      (calc_table.records.c00100 < item[1]) &
                                      (calc.records.c04470 == 0)]
        
        dictionary['Number of Non-Itemizers (m)'].append(float(val.sum()) / 1000000)
    
    # Add total rows
    for key in dictionary.keys():
        dictionary[key].append(sum(dictionary[key]))
        
    df = DataFrame.from_dict(dictionary)
    df.index = ['$0-50K', '$50K-100K', '$100K+', 'All']
    df = df.round(2)
        
    return(df)

# Begin Simulations

In [9]:
# baseline function - no policy changes, but with elasticities
def base(elast):
    """
    elast: dictionary of elasticities
    returns DF to make the table we're interested in
    """
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    # Set calculator
    recs_base = Records()
    pol_base = Policy()
    beh_base = Behavior()
    calc_base = Calculator(records=recs_base, policy=pol_base, behavior=beh_base,
                          verbose=False)
    beh_base.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_base.records.e19800 = copy.deepcopy(charitable_array)
    
    # assuming we're doing the analysis for 2017
    calc_base.advance_to_year(sim_year)
    
    # create calculator with behavioral response
    calc_base_beh = Behavior.response(calc_noref, calc_base)
    
    # Create and append DF with information interested in
    base_list.append(table(calc_base_beh))

In [10]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
def std(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std = Records()
    pol_std = Policy()
    pol_std.implement_reform(std_reform)
    beh_std = Behavior()
    calc_std = Calculator(records=recs_std, policy=pol_std, behavior=beh_std,
                          verbose=False)
    beh_std.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std.records.e19800 = copy.deepcopy(charitable_array)
    calc_std.advance_to_year(sim_year)
    
    # Create calculator with behavioral reform
    calc_std_beh = Behavior.response(calc_noref, calc_std)
    
    # create and append DF with information interested in
    std_list.append(table(calc_std_beh))

In [11]:
# ability for non-itemizers to itemize charitable deductions 
def charitable(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_charitable = Records()
    pol_charitable = Policy()
    pol_charitable.implement_reform(charitable_reform)
    beh_charitable = Behavior()
    calc_charitable = Calculator(records=recs_charitable, policy=pol_charitable, behavior=beh_charitable,
                                 verbose=False)
    beh_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_charitable.records.e19800 = copy.deepcopy(charitable_array)
    calc_charitable.advance_to_year(sim_year)
    
    # Create calculator with behaviroal reform
    calc_charitable_beh = Behavior.response(calc_noref, calc_charitable)
    
    # Create and append DF with information interested in
    charitable_list.append(table(calc_charitable_beh))

In [12]:
# decrease of the highest marginal tax rate to 35% 
def lower_mtr(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_mtr = Records()
    pol_mtr = Policy()
    pol_mtr.implement_reform(mtr_reform)
    beh_mtr = Behavior()
    calc_mtr = Calculator(records=recs_mtr, policy = pol_mtr, behavior=beh_mtr,
                          verbose=False)
    beh_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_mtr.records.e19800 = copy.deepcopy(charitable_array)
    calc_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_mtr_beh = Behavior.response(calc_noref, calc_mtr)
    
    # Create and append DF with information interested
    lower_mtr_list.append(table(calc_mtr_beh))

In [13]:
# standard deduction change and the marginal rate decrease
def std_mtr(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std_mtr = Records()
    pol_std_mtr = Policy()
    pol_std_mtr.implement_reform(std_reform)
    pol_std_mtr.implement_reform(mtr_reform)
    beh_std_mtr = Behavior()
    calc_std_mtr = Calculator(records=recs_std_mtr, policy=pol_std_mtr, behavior=beh_std_mtr,
                              verbose=False)
    calc_std_mtr.records.e19800 = copy.deepcopy(charitable_array)
    beh_std_mtr.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_mtr.advance_to_year(sim_year)
    
    # Calculator with behavioral reform
    calc_std_mtr_beh = Behavior.response(calc_noref, calc_std_mtr)
    
    # Create and append df with information interested in
    std_mtr_list.append(table(calc_std_mtr_beh))

In [14]:
# standard deduction change AND the non-itemizer deduction change
def std_charitable(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_std_charitable = Records()
    pol_std_charitable = Policy()
    pol_std_charitable.implement_reform(std_reform)
    pol_std_charitable.implement_reform(charitable_reform)
    beh_std_charitable = Behavior()
    calc_std_charitable = Calculator(records=recs_std_charitable, policy=pol_std_charitable,
                                     behavior=beh_std_charitable, verbose=False)
    calc_std_charitable.records.e19800 = copy.deepcopy(charitable_array)
    beh_std_charitable.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_std_charitable.advance_to_year(sim_year)
    
    # Calculator with behavioral response
    calc_std_charitable_beh = Behavior.response(calc_noref, calc_std_charitable)
    
    # Create and append DF with information
    std_charitable_list.append(table(calc_std_charitable_beh))

In [15]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
def all_three(elast):
    
    # Calculator with no reforms
    recs_no = Records()
    pol_no = Policy()
    calc_noref = Calculator(records=recs_no, policy=pol_no, verbose=False)
    calc_noref.records.e19800 = copy.deepcopy(charitable_array)
    calc_noref.advance_to_year(sim_year)
    
    recs_three = Records()
    pol_three = Policy()
    pol_three.implement_reform(std_reform)
    pol_three.implement_reform(charitable_reform)
    pol_three.implement_reform(mtr_reform)
    beh_three = Behavior()
    calc_three = Calculator(records=recs_three, policy=pol_three, behavior=beh_three,
                            verbose=False)
    calc_three.records.e19800 = copy.deepcopy(charitable_array)
    beh_three.update_behavior({
        sim_year: {
            '_BE_charity': [[elast['lower'], elast['middle'], elast['upper']]]
        }
    })
    calc_three.advance_to_year(sim_year)
    
    # Calculator with behavior reform
    calc_three_beh = Behavior.response(calc_noref, calc_three)
    
    # Create and append DF with information
    all_three_list.append(table(calc_three_beh))

In [16]:
# loop through each branch of elasticites, calling each function
for item in elasticities:
    base(item)
    std(item)
    charitable(item)
    lower_mtr(item)
    std_mtr(item)
    std_charitable(item)
    all_three(item)

taxcalc/behavior.py:306: RuntimeWarning: invalid value encountered in divide
  delta_income * calc.records.e00200 / agi_m_ided,
taxcalc/behavior.py:310: RuntimeWarning: invalid value encountered in divide
  delta_income * other_income / agi_m_ided,
taxcalc/behavior.py:313: RuntimeWarning: invalid value encountered in divide
  delta_income * ided / agi_m_ided,


# Output (In Millions)

Each table is in the form of a DataFrame and is ready to be exported if need be

# baseline function

In [17]:
# baseline function
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
base_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,206210.44,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,477849.84,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2108967.48,170597.52,159249.97,11347.55
All,43.83,126.59,2793027.75,284416.28,209670.41,74745.87


In [18]:
# baseline function
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
base_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,206210.44,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,477849.84,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2108967.48,170597.52,159249.97,11347.55
All,43.83,126.59,2793027.75,284416.28,209670.41,74745.87


In [19]:
# baseline function
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
base_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,206210.44,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,477849.84,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2108967.48,170597.52,159249.97,11347.55
All,43.83,126.59,2793027.75,284416.28,209670.41,74745.87


# increase value of the standard deduction

In [20]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,184714.26,58186.17,12569.10,45617.07
$50K-100K,13.46,18.94,453012.48,54004.03,36318.07,17685.96
$100K+,23.39,6.54,2091078.69,168231.83,157016.19,11215.64
All,43.83,126.59,2728805.43,280422.02,205903.35,74518.68


In [21]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,184714.26,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,453012.48,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2091078.69,170597.52,159249.97,11347.55
All,43.83,126.59,2728805.43,284416.28,209670.41,74745.87


In [22]:
# increase value of the standard deduction ($11,000 for individuals and $22,000 for joint filers,
# following the Camp proposal)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,184714.26,56973.82,11407.94,45565.88
$50K-100K,13.46,18.94,453012.48,51470.85,33944.36,17526.49
$100K+,23.39,6.54,2091078.69,165005.02,153969.30,11035.72
All,43.83,126.59,2728805.43,273449.70,199321.60,74128.09


# ability for non-itemizers to itemize charitable deductions 

In [23]:
# ability for non-itemizers to itemize charitable deductions 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
charitable_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,203486.26,59631.14,12871.63,46759.51
$50K-100K,13.46,18.94,473426.92,56838.38,37485.75,19352.62
$100K+,23.39,6.54,2104540.29,171718.92,159103.88,12615.04
All,43.83,126.59,2781453.47,288188.44,209461.26,78727.18


In [24]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
charitable_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,203634.24,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,473747.78,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2104892.58,170597.52,159249.97,11347.55
All,43.83,126.59,2782274.60,284416.28,209670.41,74745.87


In [25]:
# ability for non-itemizers to itemize charitable deductions
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
charitable_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,202977.64,63435.74,12760.86,50674.88
$50K-100K,13.46,18.94,472798.57,59917.20,37424.07,22493.13
$100K+,23.39,6.54,2104116.83,173248.52,158904.62,14343.90
All,43.83,126.59,2779893.04,296601.46,209089.54,87511.91


# decrease of the highest marginal tax rate to 35% # Lower MTR Results

In [26]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
lower_mtr_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,206210.44,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,477849.84,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2084483.73,169693.46,158345.83,11347.63
All,43.83,126.59,2768544.01,283512.22,208766.27,74745.95


In [27]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
lower_mtr_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,206210.44,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,477849.84,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2084180.59,170597.52,159249.97,11347.55
All,43.83,126.59,2768240.87,284416.28,209670.41,74745.87


In [28]:
# decrease of the highest marginal tax rate to 35% 
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
lower_mtr_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,206210.44,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,477849.84,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2084897.78,168460.33,157112.58,11347.75
All,43.83,126.59,2768958.06,282279.09,207533.02,74746.07


# standard deduction change and the marginal rate decrease

In [29]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_mtr_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,184714.26,58186.17,12569.10,45617.07
$50K-100K,13.46,18.94,453012.48,54004.03,36318.07,17685.96
$100K+,23.39,6.54,2066646.04,167334.66,156118.87,11215.80
All,43.83,126.59,2704372.78,279524.86,205006.03,74518.83


In [30]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_mtr_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,184714.26,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,453012.48,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2066344.85,170597.52,159249.97,11347.55
All,43.83,126.59,2704071.59,284416.28,209670.41,74745.87


In [31]:
# standard deduction change and the marginal rate decrease
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_mtr_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,184714.26,56973.82,11407.94,45565.88
$50K-100K,13.46,18.94,453012.48,51470.85,33944.36,17526.49
$100K+,23.39,6.54,2067057.13,162884.12,151848.04,11036.08
All,43.83,126.59,2704783.87,271328.80,197200.34,74128.45


# standard deduction change AND the non-itemizer deduction change

In [32]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
std_charitable_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,182216.23,59228.81,12768.88,46459.93
$50K-100K,13.46,18.94,446558.06,56628.55,37337.40,19291.14
$100K+,23.39,6.54,2083103.27,170544.04,158088.79,12455.24
All,43.83,126.59,2711877.55,286401.39,208195.07,78206.32


In [33]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
std_charitable_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,182321.55,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,446844.28,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2083338.18,170597.52,159249.97,11347.55
All,43.83,126.59,2712504.01,284416.28,209670.41,74745.87


In [34]:
# standard deduction change AND the non-itemizer deduction change
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
std_charitable_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,181856.94,61636.52,12301.36,49335.15
$50K-100K,13.46,18.94,446002.06,59291.91,36981.98,22309.93
$100K+,23.39,6.54,2082922.06,170471.09,156504.95,13966.14
All,43.83,126.59,2710781.06,291399.51,205788.29,85611.22


# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)

In [35]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -0.5, 'middle': -0.5, 'upper': -0.5}
all_three_list[0]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,182216.23,59228.81,12768.88,46459.93
$50K-100K,13.46,18.94,446558.06,56628.55,37337.40,19291.14
$100K+,23.39,6.54,2058790.36,169633.93,157184.27,12449.67
All,43.83,126.59,2687564.65,285491.29,207290.55,78200.74


In [36]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -1.0, 'middle': -1.0, 'upper': -1.0}
all_three_list[1]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,182321.55,58535.34,12903.53,45631.81
$50K-100K,13.46,18.94,446844.28,55283.41,37516.91,17766.51
$100K+,23.39,6.54,2058710.78,170597.52,159249.97,11347.55
All,43.83,126.59,2687876.60,284416.28,209670.41,74745.87


In [37]:
# all three changes (standard deduction increase, non-itemizer deductions, and marginal tax rate decrease)
# elasticity: {'lower': -2.236, 'middle': -1.490, 'upper': -1.182}
all_three_list[2]

,Number of Itemizers (m),Number of Non-Itemizers (m),Revenue ($m),Total Giving ($m),Total Giving: Itemizers ($m),Total Giving: Non-Itemizers ($m)
$0-50K,6.98,101.11,181856.94,61636.52,12301.36,49335.15
$50K-100K,13.46,18.94,446002.06,59291.91,36981.98,22309.93
$100K+,23.39,6.54,2059031.43,168319.60,154366.65,13952.95
All,43.83,126.59,2686890.43,289248.03,203650.00,85598.03
